In [111]:
#Imports cell
#TODO clean this up
#this notebook is VERY ROUGH, but hopefully will one day apply U-net to a database of 3d ct scans
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image
import os
import tensorflow as tf
from keras.models import Sequential, Model 
from keras.layers import *
import keras.backend as k
import keras.utils
from sklearn.model_selection import train_test_split

print(k.image_dim_ordering())

k.set_image_dim_ordering('th')

#tf.keras

th


In [140]:
#These paths are specific to my computer
#change them to match where you store your nrrds and segs
rootPath = "E:\\Patient Data Summer\\NN\\"
nrrdPath = "E:\\Patient Data Summer\\NN\\Nrrds\\"
segPath = "E:\\Patient Data Summer\\NN\\Segs\\"

nrrdFilePaths = []
nrrdFileNames = []
normalizedFilePaths = []
normalizedFileNames = []

labelFilePaths = []

#mainly unused, i could probably remove this
def loadNpy(name):
    b = np.load(name)
    return b

def bnormalize(arr):
    arrAbsolute = np.fabs(arr)
    arrMax = np.amax(arrAbsolute)
    print(arrAbsolute)
    print(arrMax)
    if arrMax != 0:
        arr = np.divide(arrAbsolute,arrMax)
        print("divided")
    else:
        print("error, max value is zero")
    return arr

for root, dirs, files in os.walk(nrrdPath):
        for filename in files:
            if filename.endswith("normalized.npy"):
                continue
            elif filename.endswith(".npy"):
                path=os.path.join(root,filename)
                print(path)
                a = loadNpy(path)
                nrrdFilePaths.append(path)
                #very messy, but is necessary to weed out the normalized images
                if filename.endswith("normalized.npy"):
                    continue
                elif filename.endswith(".npy"):
                    noExtension = filename[:-4]
                    nrrdFileNames.append(noExtension)
                    normalizedFileNames.append(noExtension)
                    newExtension = noExtension + '_normalized.npy'
                    outPutFileName = os.path.join(root,newExtension)
                    normalizedFilePaths.append(outPutFileName)
                

print(nrrdFilePaths)
print(normalizedFilePaths)
print(nrrdFileNames)



E:\Patient Data Summer\NN\Nrrds\PTMS0088_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTMS0088_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Supine.npy
['E:\\Patient Data Summer\\NN\\Nrrds\\PTMS0088_Prone.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMS0088_Supine.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMW0075_Prone.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTMW0075_Supine.npy', 'E:\\Patient Data Summer\\NN\\Nrrds\\PTOC007

In [95]:
#get labels
#I dont normalize these because they are based on binary label maps
for root, dirs, files in os.walk(segPath):
        for filename in files:
            if filename.endswith(".npy"):
                path=os.path.join(root,filename)
                labelFilePaths.append(path)
print(labelFilePaths)

['E:\\Patient Data Summer\\NN\\Segs\\PTMS0088_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMS0088_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMW0075_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTMW0075_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOC0078_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOC0078_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOD0072_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTOD0072_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPD0028_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPD0028_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPP0051_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPP0051_SupSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPR0077_ProSegLabel.npy', 'E:\\Patient Data Summer\\NN\\Segs\\PTPR0077_SupSegLabel.npy']


In [113]:
#downsize CTs
from scipy.ndimage import zoom
nnnn=0
for i in nrrdFilePaths:
    print(i)
    a = loadNpy(i)
    a = zoom(a, (0.25,0.25,0.25))
    np.save(normalizedFilePaths[nnnn],a)
    nnnn+=1

E:\Patient Data Summer\NN\Nrrds\PTMS0088_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTMS0088_Supine.npy


C:\Users\keira\.conda\envs\DL1\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


E:\Patient Data Summer\NN\Nrrds\PTMW0075_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTMW0075_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTOC0078_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTOD0072_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPD0028_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPP0051_Supine.npy
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Prone.npy
E:\Patient Data Summer\NN\Nrrds\PTPR0077_Supine.npy


In [114]:
#downsize labels
nnnnn=0
for i in labelFilePaths:
    print(i)
    a = loadNpy(i)
    a = zoom(a, (0.25,0.25,0.25))
    np.save(labelFilePaths[nnnnn],a)
    nnnnn+=1

E:\Patient Data Summer\NN\Segs\PTMS0088_ProSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTMS0088_SupSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTMW0075_ProSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTMW0075_SupSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTOC0078_ProSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTOC0078_SupSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTOD0072_ProSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTOD0072_SupSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTPD0028_ProSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTPD0028_SupSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTPP0051_ProSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTPP0051_SupSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTPR0077_ProSegLabel.npy
E:\Patient Data Summer\NN\Segs\PTPR0077_SupSegLabel.npy


In [129]:
#create the normalized images
n=0
for i in normalizedFilePaths:
    print(i)
    a = loadNpy(i)
    b = bnormalize(a)
    np.save(normalizedFilePaths[n],b)
    n+=1
print(normalizedFilePaths)

E:\Patient Data Summer\NN\Nrrds\PTMS0088_Prone_normalized.npy
[[[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 ...

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 204

E:\Patient Data Summer\NN\Nrrds\PTOC0078_Supine_normalized.npy
[[[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 ...

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 20

E:\Patient Data Summer\NN\Nrrds\PTPP0051_Prone_normalized.npy
[[[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  ...
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]]

 ...

 [[2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 2048. 2048. 2048.]
  [2048. 2048. 2048. ... 204

In [130]:
#might not be necessary, but creates a list of the actual arrays, not just the paths
nn=0
listOfNormalized=[]
for i in normalizedFilePaths:  
    a=np.load(i)
    listOfNormalized.append(a)
    nn+=1
print(listOfNormalized)

[array([[[0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        ...,
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251]],

       [[0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        ...,
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
         0.8797251],
        [0.8797251, 0.8797251, 0.8797251, ..., 0.8797251, 0.8797251,
  

In [131]:
#same as above but for labels
#please excuse the variable names
nnn=0
listOfLabels=[]
for i in labelFilePaths:  
    a=np.load(i)
    listOfLabels.append(a)
    nnn+=1
print(listOfLabels)

[array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...,

In [52]:
#mostly borrowed from an article online
#i probably wont use this in the final version, this is mainly a placeholder for testing
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = k.flatten(y_true)
    y_pred_f = k.flatten(y_pred)
    intersection = k.sum(y_true_f * y_pred_f)
    
    return (2. * intersection + smooth) / (k.sum(y_true_f) + k.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    
    return -dice_coef(y_true, y_pred)

In [107]:
X_train, X_test, y_train, y_test = train_test_split (listOfNormalized, listOfLabels, test_size = 0.2, random_state = 0)

In [139]:
#almost working fixed-layer implementation
#everything works except for the Model.fit line at the end
#I doubt it is ideal or efficient but it was more my way of playing around with the unet as i learned how it worked

NN = Model()
inp = Input((1,None,128,128))

#contracting path
conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inp)
conv1 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
drop1 = Dropout(0.5)(pool1)

print(drop1.shape)
conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(drop1)
conv2 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
drop2 = Dropout(0.5)(pool2)
print(drop2.shape)
conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(drop2)
conv3 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
drop3 = Dropout(0.3)(pool3)
print(drop3.shape)
conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(drop3)
conv4 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)
drop4 = Dropout(0.3)(pool4)
print(drop4.shape)
conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(drop4)
conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv5)
print(conv5.shape)
#expansion path
up6 = concatenate([Conv3DTranspose(256, (2, 2, 2), strides=2, padding='same')(conv5), conv4], axis=3)
conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(up6)
conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv6)
print(conv6.shape)
up7 = concatenate([Conv3DTranspose(128, (2, 2, 2), strides=2, padding='same')(conv6), conv3], axis=3)
conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(up7)
conv7 = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(conv7)
print(conv7.shape)
up8 = concatenate([Conv3DTranspose(64, (2, 2, 2), strides=2, padding='same')(conv7), conv2], axis=3)
conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(up8)
conv8 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv8)
print(conv8.shape)
up9 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=2, padding='same')(conv8), conv1], axis=3)
conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up9)
conv9 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv9)
print(conv9.shape)
conv10 = Conv3D(1, (1, 1, 3), activation='sigmoid')(conv9)
print(conv10.shape)
out = Model(inputs=[inp], outputs=[conv10])
out.compile(optimizer='rmsprop', loss=dice_coef_loss, metrics=[dice_coef])
#out.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
#Ive read the docs and I dont see any reason why the following line wouldnt work
#fit() take x and y which can both be a list of arrays
Model.fit(X_train, y_train,epochs=1,verbose=1)
#score = model.evaluate(x_test, y_test, batch_size=128)

NN.summary()

(?, 32, ?, 64, 64)
(?, 64, ?, 32, 32)
(?, 128, ?, 16, 16)
(?, 256, ?, 8, 8)
(?, 512, ?, 8, 8)
(?, 256, ?, ?, 16)
(?, 128, ?, ?, 32)
(?, 64, ?, ?, 64)
(?, 32, ?, ?, 128)
(?, 1, ?, ?, 126)


AttributeError: 'list' object has no attribute '_standardize_user_data'

In [117]:
ctSize=128

In [136]:
#Adapted from segmentation2 to work for 3d
#not yet working

# Construct a U-net model

num_classes = 2
filter_multiplier = 8

def nvidia_unet(patch_size=ctSize, num_classes=num_classes):
    input_ = Input((32, None, patch_size, patch_size))
    skips = []
    output = input_
    
    num_layers = int(np.floor(np.log2(patch_size)))
    down_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    down_filter_numbers = np.zeros([num_layers], dtype=int)
    up_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    up_filter_numbers = np.zeros([num_layers], dtype=int)
    
    for layer_index in range(num_layers):
        down_conv_kernel_sizes[layer_index] = int(3)
        down_filter_numbers[layer_index] = int( (layer_index + 1) * filter_multiplier + num_classes )
        up_conv_kernel_sizes[layer_index] = int(4)
        up_filter_numbers[layer_index] = int( (num_layers - layer_index - 1) * filter_multiplier + num_classes )
    
    print("Number of layers:       {}".format(num_layers))
    print("Filters in layers down: {}".format(down_filter_numbers))
    print("Filters in layers up:   {}".format(up_filter_numbers))
    
    for shape, filters in zip(down_conv_kernel_sizes, down_filter_numbers):
        skips.append(output)
        output= Conv3D(filters, (shape, shape, shape), strides=2, padding="same", activation="relu")(output)
    
    for shape, filters in zip(up_conv_kernel_sizes, up_filter_numbers):
        output = keras.layers.UpSampling3D(data_format='channels_first')(output)
        skip_output = skips.pop()
        #the next line is throwing an error, not sure why
        print(output.shape)
        print(skip_output.shape)
        output = concatenate([Conv3DTranspose(filters, (2,2,2), strides=2, padding='same')(output), skip_output], axis=3)
        if filters != num_classes:
            activation = "relu"
            output = Conv3D(filters, (shape, shape, shape), activation="relu", padding="same")(output)
            output = BatchNormalization(momentum=.9)(output)
        else:
            activation = "softmax"
            output = Conv3D(filters, (shape, shape, shape), activation="softmax", padding="same")(output)
    
    assert len(skips) == 0
    return Model([input_], [output])

model = nvidia_unet(ctSize, num_classes)

# model.summary()

Number of layers:       7
Filters in layers down: [10 18 26 34 42 50 58]
Filters in layers up:   [50 42 34 26 18 10  2]
(?, 58, ?, 2, 2)
(?, 50, ?, 2, 2)


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 50, None, 4, 4), (None, 50, None, 2, 2)]

In [ ]:
max_learning_rate = 0.0002
min_learning_rate = 0.000002
num_epochs = 15

learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs

model.compile(optimizer=keras.optimizers.adam(lr=max_learning_rate, decay=learning_rate_decay),
               loss= "binary_crossentropy",
               metrics=["accuracy"])

print("Learning rate decay = {}".format(learning_rate_decay))

Model.fit(X_train,
          validation_data=y_train,
          epochs=num_epochs,
          verbose=1)
